In [2]:
import pandas as pd
import numpy as np
import os

<u>Reminder</u>:
* **DPD:**	Dominant wave period (seconds) is the period with the maximum wave energy. See the Wave Measurements section.
* **APD:**	Average wave period (seconds) of all waves during the 20-minute period. See the Wave Measurements section.

In [21]:
dataMain = '/home/madhatter106/DATA/Beaching_Project/CapeCodData/'
pklDir = os.path.join(dataMain, 'PklJar')
df=pd.read_pickle(os.path.join(pklDir,'dfAgg.pkl'))
df.rename(columns={'EasterlyPersistence': 'EasterlyWS',
                   'SoutherlyPersistence': 'SoutherlyWS'}, inplace=True)

In [20]:
df.head()

,APD,ATMP,BAR,DPD,EasterlyWD,GST,SoutherlyWD,WD,WSPD,WTMP,WVHT
date,,,,,,,,,,,
1999-01-01 00:00:00,5.25,-4.3,1017.4,11.11,-3.542719,7.2,-4.075432,221.0,5.4,5.6,0.33
1999-01-01 01:00:00,5.51,-4.0,1016.5,11.11,-3.447704,7.3,-4.412860,218.0,5.6,5.7,0.31
1999-01-01 02:00:00,6.53,-3.9,1015.8,12.50,-4.100237,7.0,-3.959553,226.0,5.7,5.7,0.32
1999-01-01 03:00:00,6.17,-3.9,1015.3,11.11,-4.235926,7.3,-3.814044,228.0,5.7,5.7,0.31
1999-01-01 04:00:00,5.02,-3.8,1015.0,11.11,-4.948156,7.7,-3.213370,237.0,5.9,5.6,0.39


In [22]:
dfVars = ['EasterlyWS', 'SoutherlyWS', 'WTMP', 'WVHT', 'APD', 'DPD']
dfDailyMean = df[dfVars].resample('D').mean()

In [23]:
dfDailyMean.head()

,EasterlyWS,SoutherlyWS,WTMP,WVHT,APD,DPD
date,,,,,,
1999-01-01,-8.369086,-1.962736,5.629167,0.782083,4.260000,7.372083
1999-01-02,-10.358338,2.124278,5.470833,1.029167,3.723333,4.049167
1999-01-03,5.530758,-3.071964,5.187500,1.402083,3.920000,4.574583
1999-01-04,-6.726604,-2.535160,5.108333,1.735000,6.136667,10.669583
1999-01-05,-8.472319,-0.380683,4.904348,0.875000,4.175455,8.880909


In [24]:
dfDailyMean.to_pickle('../PklJar/dfDailyWind.pkl')

In [26]:
def MakeMonthlyIndexRelatedDF(df):
    indexbymonth = df[dfVars].groupby(df.index.month).mean()
    dfM = df.resample('M').mean()
    
    dfIndex = pd.DataFrame(index=dfM.index, 
                       columns=dfM.columns, dtype='f8')
    for i in indexbymonth.index:
        dfIndex.loc[dfM.index.month==i]=dfM.loc[dfM.index.month==i].subtract(indexbymonth.loc[i])
    dfIndex.rename(columns={'EasterlyWS': 'EasterlyWSIndex',
                            'SoutherlyWS': 'SoutherlyWSIndex'})
    return dfIndex

In [31]:
dfMoIndex = MakeMonthlyIndexRelatedDF(df)

In [32]:
dfMoIndex.drop(['ATMP', 'BAR', 'GST', 'WD', 'WSPD'], axis=1, inplace=True)

In [33]:
dfMoIndex.to_pickle('../PklJar/dfMonthlyWSIndex.pkl')